<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisCuantitativoAvanzado/blob/main/Ejercicio_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 7

## Cargar la base

La siguiente base describe datos sobre los salarios, la formación y la experiencia de empleadores que realizan entrevistas de trabajo.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d mohithsairamreddy/salary-data

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Mounted at /content/drive
salary-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [ ]:
## Cargamos la base
df = pd.read_csv('/content/Salary_Data.csv')
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


## Procesamiento de la base

Vemos que la variable `Education Level` tiene varios valores similares escritos de manera distinta.

In [ ]:
df["Education Level"].value_counts()

Bachelor's Degree    2267
Master's Degree      1573
PhD                  1368
Bachelor's            756
High School           448
Master's              288
phD                     1
Name: Education Level, dtype: int64

Para unificar estos valores, la recatogrizamos creando una nueva variable denominada `Education New`:

In [ ]:
df["Education New"]=df["Education Level"].copy()

In [ ]:
df["Education New"].loc[df["Education Level"].str.contains("Bach")==True]="Bachelor's"
df["Education New"].loc[df["Education Level"].str.contains("Mast")==True]="Master's"
df["Education New"].loc[df["Education Level"].str.contains("hD")==True]="PhD"


<ipython-input-18-42d4324270b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Education New"].loc[df["Education Level"].str.contains("Bach")==True]="Bachelor's"
<ipython-input-18-42d4324270b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Education New"].loc[df["Education Level"].str.contains("Mast")==True]="Master's"
<ipython-input-18-42d4324270b5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Education New"].loc[

Verificamos que con estos cambios, las categorías quedaron unificadas

In [ ]:
df["Education New"].value_counts()

Bachelor's     3023
Master's       1861
PhD            1369
High School     448
Name: Education New, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6704 entries, 0 to 6703
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6702 non-null   float64
 1   Gender               6702 non-null   object 
 2   Education Level      6701 non-null   object 
 3   Job Title            6702 non-null   object 
 4   Years of Experience  6701 non-null   float64
 5   Salary               6699 non-null   float64
 6   Education New        6701 non-null   object 
dtypes: float64(3), object(4)
memory usage: 366.8+ KB


Borramos además los datos faltantes de la base

In [ ]:
df=df.dropna()

Verificamos que ahora no quedan datos faltantes

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6698 entries, 0 to 6703
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6698 non-null   float64
 1   Gender               6698 non-null   object 
 2   Education Level      6698 non-null   object 
 3   Job Title            6698 non-null   object 
 4   Years of Experience  6698 non-null   float64
 5   Salary               6698 non-null   float64
 6   Education New        6698 non-null   object 
dtypes: float64(3), object(4)
memory usage: 418.6+ KB


# Preguntas

## Pregunta 1

Realizar un test de hipótesis para ver si la proporción de mujeres es significativamente menor al 50%.

*Ayuda:* el atributo `.shape` devuelve la cantidad de filas y columnas de una base. Por lo tanto, tomando `df.shape[0]` se calcula la cantidad de filas de la base `df` y por lo tanto, la cantidad de individuos.
Para contar la cantidad de mujeres, por ejemplo, se puede recorrer todas las filas de "Gender" viendo en cada instancia si el valor coincide con "Female". Cuando se suman esos valores, se obtiene la cantidad de mujeres: `np.sum(df["Gender"]=="Female")`. El valor de referencia sería `value=0.5` y determinar la hipótesis alternativa correspondiente. Utilizar el `help` para ver cómo funciona el comando.


In [ ]:
from statsmodels.stats.proportion import proportions_ztest


In [ ]:
help(proportions_ztest)

## Pregunta 2

Notemos que el porcentaje identificado como `Other` en la base es pequeño y no es un número que permita obtener conclusiones estadísticas pertinentes.

Realizar un test de hipótesis para ver si el salario medio de los hombres es **mayor** que el de las mujeres, según estos datos.

*Ayuda:* Se están evaluando medias, no porcentajes, por lo que se usa la función provista. Para comparar las medias, se deben pasar dos conjuntos de datos como argumento. Para identificar, por ejemplo, los datos de salarios sólo de las mujeres, se puede utilizar el siguiente comando: `df.Salary[df["Gender"]=="Female"]`

In [ ]:
from statsmodels.stats.weightstats import ztest


In [ ]:
help(ztest)

## Pregunta 3

Realizar un test de ANOVA para ver si el salario medio cambia con los distintos niveles educativos. Utilizar la comparación múltiple para ver cuáles son diferentes entre sí

*Ayuda:* Los datos deben pasarse en una **lista**. Entonces, habría que definir previamente los datos de salario para cada nivel educativo. Para identificar el salario por cada nivel educativo, se puede usar la ayuda anterior reemplazando "Gender" por "Education New" y "Female" por "High School" para obtener los salarios de personas con título secundario completo. Para la comparación múltiple, se pueden usar directamente las columnas `df["Salary"]` (datos) y `df["Education New"]` (grupos).

In [ ]:
from statsmodels.stats.oneway import anova_oneway
from statsmodels.sandbox.stats.multicomp import MultiComparison

In [ ]:
# Datos de salarios de individuos con título secundario completo
x1=
# Datos de salarios de individuos con título de grado completo
x2=
# Datos de salarios de individuos con título de maestría completo
x3=
# Datos de salarios de individuos con título de doctorado completo
x4=
print(anova_oneway([x1,x2,x3,x4]))
MC=MultiComparison()
print(MC.tukeyhsd())

In [ ]:
help(anova_oneway)
help(MultiComparison)

## Pregunta 4

Testear si la variable de años de experiencia tiene distribución normal y obtenga alguna conclusión. Hacer un histograma de la variable para comparar con los resultados del test.



In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import shapiro

In [ ]:
help(shapiro)
help(plt.hist)

## Pregunta 5

Testear si hay correlación significativa entre las variables de años de experiencia y el salario. Hacer un gráfico de dispersión para contrastar con el resultado provisto por el test.



In [ ]:
from scipy.stats import pearsonr

In [ ]:
help(pearsonr)
help(plt.scatter)

In [ ]:
from statsmodels.stats.weightstats import zconfint